In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
df_consolidado = []

for file in os.listdir():
    if file.endswith('.xlsx'):
        print(f'Transforming: {file}')
        tmp_df = pd.read_excel(file)

        tmp_df.rename(
            columns={
                'Entrada/Saída' : 'entrada_saida',
                'Data' : 'data',
                'Movimentação' : 'des_movimentacao',
                'Produto' : 'produto',
                'Instituição' : 'instituicao',
                'Quantidade' : 'quantidade', 
                'Preço unitário' : 'preco_un', 
                'Valor da Operação' : 'preco_total'
            }, 
            inplace=True
        )

        tmp_df['des_movimentacao'] = tmp_df['des_movimentacao'].str.lstrip().str.rstrip()
        tmp_df['produto'] = tmp_df['produto'].str.lstrip().str.rstrip()
        tmp_df['instituicao'] = tmp_df['instituicao'].str.lstrip().str.rstrip()

        des_movimentacao_dividendos = ['Juros Sobre Capital Próprio', 'Rendimento', 'Dividendo']
        des_movimentacao_venda = ['Transferência - Liquidação']

        tmp_df['tp_movimentacao'] = np.where(
            tmp_df['des_movimentacao'].isin(des_movimentacao_dividendos), 'Dividendos',
            np.where(
                tmp_df['des_movimentacao'].isin(des_movimentacao_venda), 'Venda', tmp_df['des_movimentacao']
            )
        )

        tp_movimentacao_entrada = ['Dividendos', 'Venda', 'Rendimento', 'Leilão de Fração', 'Juros']

        tmp_df['entrada_saida'] = np.where(
            tmp_df['tp_movimentacao'].isin(['Compra', 'Cobrança de Taxa Semestral']), 'Saida',
                np.where(
                    tmp_df['tp_movimentacao'].isin(tp_movimentacao_entrada), 'Entrada', np.nan
                )
        )  

        fii = ['BTLG11', 'KNRI11']

        acao = [
            'RAPT4 ', 'TRPL4', 'HYPE3', 'ITSA4', 
            'NUBR33', 'BBSE3', 'BRAP3', 'BBDC4', 
            'ITUB4',  'MGLU3', 'CPFE3', 'CMIG4', 
            'TCNO4',  'CSMG3', 'OIBR4', 'ENAT3', 
            'TAEE11', 'PETR4', 'SOMA3 '
        ]

        tmp_df['categoria'] = np.where(
            tmp_df['produto'].str.contains('Tesouro'), 'Tesouro Direto',
            np.where(
                tmp_df['produto'].isin(fii), 'FII',
                np.where(
                    tmp_df['produto'].isin(acao), 'Ação', np.nan
                )
            )
        )        

        df_consolidado.append(tmp_df)     

df_consolidado = pd.concat(df_consolidado)

In [ ]:
df_consolidado[df_consolidado['entrada_saida'] == 'nan']